# Homework 3: Conditions, loops, and functions
ENVR 890-001: Python for Environmental Research, Fall 2020

By Andrew Hamilton. 

### Instructions
For this assignment, we will build a simulation model to study the spread of a contagious disease such as SARS-CoV-2. This can be thought of as an [agent-based](https://en.wikipedia.org/wiki/Agent-based_model) version of the classic SIRD (susceptible-infectious-recovered-deceased) model. Major caveat: **I am not an epidemiologist and this model is highly simplified**. This simulation is based on the [excellent video](https://www.youtube.com/watch?v=gxAaO2rsdIs) by 3Blue1Brown, which you can check out for more context and more complexity.

**Due date: Sep. 4, before class**

Our simulation model will work as follows:
1. We start with $N$ people initially scattered randomly within a box, with each side having length 1
1. This population is randomly split into $I_0$ infected people and $S_0$ susceptible people. We assume no one has recovered or died at time $t_0$
1. At each time step, each individual takes a random movement from their current location, based on sampling from a normal distribution with mobility parameter $m$
1. Each infected person has a probability $p_i$ of infecting any uninfected person that is within a radius of $r$
1. Each infected person has a probability $p_r$ of recovering, $p_d$ of dying, and $1-p_r-p_d$ of remaining infected
1. We will simulate the dynamics of $S$, $I$, $R$, and $D$ over $T$ time steps

I will build out much of the model. **Your job will be to fill in the incomplete code blocks to get the final working model. All incomplete blocks are referenced with "## TODO".**

**Don't forget to save a new copy of this notebook with your last name. Resave when you are finished and email it to me.**

### Set up
First we need to make sure our notebook is working from the proper "working directory", and create a folder to hold our figures

In [2]:
## get current working directory
import os
wd = os.getcwd()
print(wd)

C:\Users\Andrew\Documents\Teaching\ENVR-890-001-Python-For-Environmental-Research-F20\HW3_FunctionsLoops


In [3]:
# ## if wd is not where you want it to be (the location of this notebook), you can set it 
# wd = 'your_wd_here'
# os.chdir(wd)

In [4]:
## Create a directory to hold figures
os.makedirs(wd + '/figs/', exist_ok = True)

Now we set up the known parameters (*I made up values that produce an interesting simulation, not based on any physical data. You can play around with the parameters to see how they affect the results!*)

In [5]:
## User-set parameters
N = 100
I0 = 10
m = 0.01
pi = 0.25
r = 0.06
pr = 0.015
pd = 0.01
T = 200

## Calculate other parameters we know based on the user-set parameters above
S0 = N - I0
R0 = 0
D0 = 0

Set up lists to hold $S$, $I$, $R$, and $D$ counts on each day of the simulation, and lists to hold coordinates and SIRD status of each person on each day. 

In [10]:
## Set up one list for each population (S, I, R, D). We will append a new value each day based on the updated population.
## Initial populations based on parameters above.
S = [S0]
I = [I0] 
R = [R0]
D = [D0]

## set up list of lists for coordinates of each person. Each list has N lists (one for each person). 
## We will append the new coordinates at each time step based on random movements. 
## Initialize with -1, a value that doesn't make sense. We will set the initial potitions soon using randomly drawn coordinates.
x = [[-1] * N]
y = [[-1] * N]

## set up list of lists for SIRD status of each person. Set all to S initially, and we will randomly change some to I soon.
SIRD = [['S'] * N]

### Initializing the starting state of the model
We will use the ``random`` module to randomly sample the initial locations of each person within the box, and the SIRD status of each individual. Refer to the [module documentation](https://docs.python.org/3/library/random.html) for help with its functionality.

In [ ]:
## import random module to for random sampling 
import random

## set "seed" so we will all get the same "random" numbers
random.seed(101)

In [ ]:
### random.uniform(0, 1) generates a random number between 0 and 1. It will be different each time it is run. 
## Uncomment the next two lines to try it. Run multiple times to get different numbers. 
# sample = random.uniform(0, 1)
# print(sample)

In [ ]:
## TODO: Now build a for loop to fill in each person's initial values of x and y with random coordinates within the box.
## Hint: we want to replace the -1's in the x and y vectors. All values should be between 0 and 1.




In [ ]:
## random.sample(list, n) will choose n random elements from list, without replacement. We will use this to choose I0 people to infect initially.
indexes = list(range(N))
initial_infections = random.sample(indexes, I0)
# print(initial_infections)

In [ ]:
## TODO: Now use a for loop to set the elements of SIRD corresponding to the indexes in initial_infections, to 'I'




In [ ]:
## The last step of initiallization is to set the numbers of S, I, R, and D at a particular time step. We will use a function to do this.
def count_SIRD(SIRD, t):
    St, It, Rt, Dt = 0, 0, 0, 0
    for i in range(N):
        if SIRD[t][i] == 'S':
            St += 1
        ## TODO: Use elif/else to write the rest of the conditions

        
        
        
    return (St, It, Rt, Dt)

In [ ]:
St, It, Rt, Dt = count_SIRD(SIRD, 0)
S.append(St)
I.append(It)
R.append(Rt)
D.append(Dt)

### Checking the initial state
Use the following function to visualize the different populations at any time step. For the initial population, they should be randomly spread around the box and mixed between susceptible and infected. Don't worry too much about how this works yet, we will learn about visualization in a couple of weeks.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

## colors from ColorBrewer
# colors = ['#a6cee3', '#1f78b4', '#b2df8a', '#33a02c']
colors = ['y', 'r', 'b', 'k']


## plotting function
def plot_locations(x, y, SIRD, t, output_plot=True, save_plot=False):
    ## get the data for the day we want to plot
    xt = x[t]
    yt = y[t]
    SIRDt = SIRD[t]
    ## set up the plot
    fig, ax = plt.subplots(figsize=(10,10))
    ax.set_aspect(1)    
    ## get colors basesd on SIRDt. Set S color as default.
    c = [colors[0]] * N
    for i in range(N):
        if SIRDt[i] == 'I':
            c[i] = colors[1]
        elif SIRDt[i] == 'R':
            c[i] = colors[2]
        elif SIRDt[i] == 'D':
            c[i] = colors[3]
    scatter = ax.scatter(xt, yt, color = c, label = SIRDt, alpha = 0.6)    
    ax.plot((0,1), (0,0), color = 'grey', ls = '--')
    ax.plot((0,1), (1,1), color = 'grey', ls = '--')
    ax.plot((0,0), (0,1), color = 'grey', ls = '--')
    ax.plot((1,1), (0,1), color = 'grey', ls = '--')
    ax.set_xlim(-0.05, 1.12)
    ax.set_ylim(-0.05, 1.05)
    ax.set_xticks((0,1))
    ax.set_yticks((0,1))
    ax.set_title('Population at t = ' + str(t))
    ## add a legend
    legend_elements = [Line2D([0], [0], marker='o', color='w', label='S', markerfacecolor=colors[0], markersize=8, alpha = 0.6),
                       Line2D([0], [0], marker='o', color='w', label='I', markerfacecolor=colors[1], markersize=8, alpha = 0.6),
                       Line2D([0], [0], marker='o', color='w', label='R', markerfacecolor=colors[2], markersize=8, alpha = 0.6),
                       Line2D([0], [0], marker='o', color='w', label='D', markerfacecolor=colors[3], markersize=8, alpha = 0.6)]
    ax.legend(handles=legend_elements, loc='right')
    ## output plot to screen?
    if not output_plot:
        plt.close()
    if save_plot:
        fig.savefig(save_plot)
    return fig

In [ ]:
fig = plot_locations(x, y, SIRD, 0)

### Function for random movements
Now we want to build a function to generate a random movement for each person. We will assume that the person moves by a normally distributed amount in both the x and y direction. The mean of the distribution is zero and the standard deviation is set by the mobility parameter $m$. If a move will take the individual outside the box, we throw it away and try again. This function uses **recursion**, meaning that it can call itself. This means that the function will keep getting called until it generates a "legal" move that is inside the box

In [ ]:
## function for generating new coordinates for an individual
def get_new_coords(x_indiv, y_indiv):
    ## the normal.gauss(mean, std) function generates a normally distributed variable
    jump_x = random.gauss(0, m)
    jump_y = random.gauss(0, m)
    ## add the jump to the old coordinates
    new_x = x_indiv + jump_x
    new_y = y_indiv + jump_y
    ## check if the new coordinates are inside the box
    if (new_x >= 0) and (new_x <=1) and (new_y >= 0) and (new_y <= 1):
#         print (x_indiv, jump_x, new_x)
        return (new_x, new_y)
    ## if not, recall the function
    else:
        return get_new_coords(x_indiv, y_indiv)

### Function for infections, recoveries, and deaths in each time step
First, we build a function to get the distance between two individuals, using the formula for Euclidean distance

In [ ]:
## function for getting distance between two individuals
## TODO: fill in this function using Euclidean distance formula. xt and yt will be lists of x and y coords at time step t, 
## and index1-2 will be the indexes of the two individuals we are interested in.
def get_distance(xt, yt, index1, index2):

    return distance

Now we want a function to probabilistically infect a susceptible person who is within the radius $r$ of an infected person. This will take as its argument the probability of infection, $p_i$, and return the new status of the susceptible person: either infected ("I") or still susceptible ("S").

In [ ]:
## function for choosing whether a susceptible person is infected by a nearby infectious person . output their new status.
def infect_or_no(pi):
    new_status = random.choices(('I', 'S'), weights = (pi, 1 - pi), k = 1)[0]
    return new_status

Similarly, we need a function to probabilistically determine if an infected person will recover or die. It should take as its arguments the probabiliites $p_r$ and $p_d$, and return the new status of the individual: recovered ("R"), dead ("D"), or still infected ("I").

In [ ]:
## TODO: finish this function
## function for an infected person to recover or die
def recover_or_die_or_no(pr, pd):

    return new_status

Now we will build a function for carrying out all of the interactions of an 

In [ ]:
## function for probabilistically infecting neighbors, and having infected people recover or die. function will return a new updated SIRD vector.
import math
def infectious_interactions(SIRDt, SIRDtplus1, xt, yt, n):
    ## first check if nth person is infected in time step t
    if SIRDt[n] == 'I':
        ## now go through all other individuals, and check if each is susceptible, and has not already been infected for next time step (by another infected person)
        for i in range(N):
            if (SIRDt[i] == 'S') and (SIRDtplus1[i] == 'S'):
                ## if so, check the distance away
                distance = get_distance(xt, yt, n, i)
                if distance <= r:
                    ## if the person is within the radius of infection, they have the probability of getting infected. choose their new status randomly.
                    SIRDtplus1[i] = infect_or_no(pi)
        ## after potentially infecting neighbors, does this infectious person recover, die, or neither
        SIRDtplus1[n] = recover_or_die_or_no(pr, pd)

    return SIRDtplus1

In [ ]:
## Function for stepping the model forward by one time step
def step_model(t):
    ## add new list for coords and SIRD status
    x.append([])
    y.append([])
#     SIRD.append([])
    ## update coords
    for n in range(N):
        x_new, y_new = get_new_coords(x[t-1][n], y[t-1][n]) 
        x[t].append(x_new)
        y[t].append(y_new)
    ## update SIRD status
    SIRDtplus1 = SIRD[t - 1].copy()
    for n in range(N):
        ## function to potentially infect neighbors and also potentially recover or die for each infectious person
        SIRDtplus1 = infectious_interactions(SIRD[t - 1], SIRDtplus1, x[t], y[t], n)
    SIRD.append(SIRDtplus1)
    ## update population counts
    St, It, Rt, Dt = count_SIRD(SIRD, t)
    S.append(St)
    I.append(It)
    R.append(Rt)
    D.append(Dt)

In [ ]:
for t in range(1, T):
    step_model(t)

In [ ]:
## Output first and last time steps to screen, and save all time steps as figs to disk
for t in range(T):
    if t in [0, T-1]:
        output_plot = True
    else:
        output_plot = False
    save_plot = wd + '/figs/t' + str(t) + '.png'
    fig_start = plot_locations(x, y, SIRD, t, output_plot = output_plot, save_plot = save_plot)

In [ ]:
## Stitch together still images from each time step to create a gif (code borrowed from https://stackoverflow.com/questions/753190/programmatically-generate-video-or-animated-gif-in-python)
import imageio
with imageio.get_writer(wd + '/figs/infection_spread.gif', mode='I', duration=0.1) as writer:
    for t in range(T):
        filename = wd + '/figs/t' + str(t) + '.png'
        image = imageio.imread(filename)
        writer.append_data(image)

Here is our gif of the population over the whole simulation. **Note: your browser may continue to show an old version if you change the parameters and rerun this. Saving your work (``Ctrl``+``s``) and then refreshing the browser should make it switch to the latest version of the gif file.**
<img src="figs/infection_spread.gif" style="width: 700px;" />

In [ ]:
## function for plotting population counts
def plot_populations(S, I, R, D):
    plt.figure(figsize=(10,10))
    plt.plot(S, color=colors[0], label='S')
    plt.plot(I, color=colors[1], label='I')
    plt.plot(R, color=colors[2], label='R')
    plt.plot(D, color=colors[3], label='D')
    plt.xlabel('Time')
    plt.ylabel('Number of individuals')
    plt.title('Population counts over time')
    plt.ylim([0, 100])
    plt.legend()
    
plot_populations(S, I, R, D)